SECOND PART: CREATE DF WITH GEOCODER

0.- Get the table as in FIRST PART (same code)

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
wikipedia='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia= requests.get(wikipedia).text
soup = BeautifulSoup(raw_wikipedia,'xml')
column_names = ['Postalcode','Borough','Neighborhood']
df_toronto = pd.DataFrame(columns = column_names)
content = soup.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    df_toronto = df_toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)
df_toronto = df_toronto[df_toronto.Borough!='Not assigned']
df_toronto = df_toronto[df_toronto.Borough!= 0]
df_toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,df_toronto.shape[0]):
    if df_toronto.iloc[i][2] == 'Not assigned':
        df_toronto.iloc[i][2] = df_toronto.iloc[i][1]
        i = i+1
                                 
df_toronto = df_toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_toronto = df_toronto.dropna()
empty = 'Not assigned'
df_toronto = df_toronto[(df_toronto.Postalcode != empty ) & (df_toronto.Borough != empty) & (df_toronto.Neighborhood != empty)]
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
df_toronto = df_toronto.groupby(['Postalcode', 'Borough'])
df_toronto = df_toronto.apply(neighborhood_list).reset_index(name='Neighborhood')
df_toronto.head()


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


1.- Install geocoder (we will use the python package)

In [2]:
!pip install geocoder
import geocoder

2.- Defining a f(x) to retrieve latitude and longitude

In [5]:
def get_coord(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords


3.- Using f(x) to create df of long and lat

In [9]:
postcodes = df_toronto['Postalcode']    
longlat = [get_coord(postal_code) for postal_code in postcodes.tolist()]
df_longlat = pd.DataFrame(longlat, columns=['Latitude', 'Longitude'])
df_longlat.head()

,Latitude,Longitude
0,43.811650,-79.195561
1,43.785605,-79.158701
2,43.765690,-79.175299
3,43.768216,-79.217610
4,43.769608,-79.239440


4.- Merging dfs

In [11]:
df_toronto['Longitude'] = df_longlat['Longitude']
df_toronto['Latitude'] = df_longlat['Latitude']
df_toronto.head(12)

,Postalcode,Borough,Neighborhood,Longitude,Latitude
0,M1B,Scarborough,"Rouge, Malvern",-79.195561,43.811650
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",-79.158701,43.785605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",-79.175299,43.765690
3,M1G,Scarborough,Woburn,-79.217610,43.768216
4,M1H,Scarborough,Cedarbrae,-79.239440,43.769608
5,M1J,Scarborough,Scarborough Village,-79.232172,43.743085
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",-79.263670,43.726260
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",-79.284910,43.713213
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",-79.234976,43.723575
9,M1N,Scarborough,"Birch Cliff, Cliffside West",-79.260069,43.696690


5.- To csv

In [12]:
df_toronto.to_csv('toronto.csv')